<div style="background:#2b6684;color:white; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:30px;color:white"><u>This Week Questions</u>
    
<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Question 1:</u></p>
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>What is the most frequent observation (mode) for the column 'neighbourhood_group'?</li>
</ul>   

<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Question 2:</u></p>
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Create the correlation matrix for the numerical features of your train dataset. </li>
<ul>
<li>In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset. </li>    
</ul> 
<li>What are the two features that have the biggest correlation in this dataset? </li>   
    
</ul>     
    

<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Question 3:</u></p>
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Calculate the mutual information score with the (binarized) price for the two categorical variables that we have. Use the training set only.</li>
<li>Which of these two variables has bigger score?</li>
<li>Round it to 2 decimal digits using round(score, 2)</li>
</ul> 
    
<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Question 4:</u></p>
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Now let's train a logistic regression</li>
<li>Remember that we have two categorical variables in the data. Include them using one-hot encoding.</li>
<li>Fit the model on the training dataset.</li>
<ul>
<li>To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:</li> 
<li>model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)</li>
</ul>     
<li>Calculate the accuracy on the validation dataset and rount it to 2 decimal digits.</li>
</ul>    
    
    
<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Question 5:</u></p>
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>We have 9 features: 7 numerical features and 2 categorical.</li>
<li>Let's find the least useful one using the feature elimination technique.</li>
<li>Train a model with all these features (using the same parameters as in Q4).</li>
<li>Now exclude each feature from this set and train a model without it. Record the accuracy for each model.</li>
<li>For each feature, calculate the difference between the original accuracy and the accuracy without the feature.</li>
<li>Which of following feature has the smallest difference? </li>
<ul>
<li>neighbourhood_group</li> 
<li>room_type</li>
<li>number_of_reviews</li> 
<li>reviews_per_month</li>
</ul> 
</ul>      

<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Question 6:</u></p>
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>For this question, we'll see how to use a linear regression model from Scikit-Learn</li>
<li>We'll need to use the original column 'price'. Apply the logarithmic transformation to this column.</li>
<li>Fit the Ridge regression model on the training data.</li>
<li>This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]</li>
<li>Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.</li>
</ul>

</div>


<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Importing Libraries</div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.metrics import mutual_info_score, mean_squared_error
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression, Ridge

plt.rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings("ignore")

<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Loading and Reading Data </div>

In [2]:
data = pd.read_csv('../input/new-york-city-airbnb-open-data/AB_NYC_2019.csv')
data.head()



<div style="background:#2b6684;color:white; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Features used for this Project</div>

In [3]:
features  = [
    'neighbourhood_group',
    'room_type',
    'latitude',
    'longitude',
    'price',
    'minimum_nights',
    'number_of_reviews',
    'reviews_per_month',
    'calculated_host_listings_count',
    'availability_365'
]
df = data[features]
"""Checking description of this project's data set"""
df.describe()

<div style="background:#2b6684;color:white; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Missing Values? Impute them with 0</div>

In [4]:
from IPython.display import display
missing_vals = df.isnull().sum()
print("Before Imputing Missing Values")
display(missing_vals.to_frame().reset_index().rename({'index': 'Variables', 0: 'Missing Values'}, axis = 1).sort_values(by = 'Missing Values', ascending = False).style.background_gradient('Reds'))


df.fillna(0, inplace = True)
print("After Imputing Missing Values")
display(df.isnull().sum().to_frame().reset_index().rename({'index': 'Variables', 0: 'Missing Values'}, axis = 1).style.background_gradient('Reds'))


<div style="background:#2b6684;color:white; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:white">Q1. What is the most frequent observation (mode) for the column 'neighbourhood_group'?</div>

In [5]:
print("Mode for variable 'neighbourhood_group': %s" %(df['neighbourhood_group'].value_counts().head(1)))

<div style="background:#2b6684;color:white; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Split the data
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Split your data in train/val/test sets, with 60%/20%/20% distribution.</li>
<li>Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.</li>
<li>Make sure that the target value ('price') is not in your dataframe.</li>
</ul>
</div>


In [6]:
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)
len(df_train), len(df_val), len(df_test)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


<div style="background:#2b6684;color:white; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Question 2:
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Create the correlation matrix for the numerical features of your train dataset. </li>
<ul>
<li>In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset. </li>    
</ul> 
<li>What are the two features that have the biggest correlation in this dataset? </li>   
    
</ul> 
</div>


In [7]:
"""Creating List of Numerical and Categorical columns"""
categorical = [col for col in df.columns if df[col].dtype == 'object']
numerical = [col for col in df.columns if col not in categorical]

"""Correlation of Numerical Columns"""
display(df[numerical].corr())

"""Heatmap of Numerical Variables"""
display(sns.heatmap(df[numerical].corr(), annot = True, lw = 0.2, square = True, cmap = 'crest'))


<div style="background:#2b6684;color:white; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Highest Correlation is between
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>reviews_per_month and number_of_reviews: 0.589407</li>
</ul>
</div>

<div style="background:#2b6684;color:white; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Make price binary
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>We need to turn the price variable from numeric into binary.</li>
<li>Let's create a variable above_average which is 1 if the price is above (or equal to) 152.</li>
</ul>
</div>

In [8]:
"""price variable from numeric into binary."""
df_full_train['above_average'] = (df_full_train['price'] >= 152).values.astype(int)


<div style="background:#2b6684;color:white; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Question 3:
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Calculate the mutual information score with the (binarized) price for the two categorical variables that we have. Use the training set only.</li>
<li>Which of these two variables has bigger score?</li>
<li>Round it to 2 decimal digits using round(score, 2)</li>
</ul> 
</div>


In [9]:
"""Mutual Information"""
def mutual_info_bin_score(series):
    return mutual_info_score(series, df_full_train.above_average)

mi = df_full_train[categorical].apply(mutual_info_bin_score)
mi.round(2).sort_values(ascending=False)    

<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:white">Variable room_type has the biggest score = 0.14</div>

<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Question 4:
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Now let's train a logistic regression</li>
<li>Remember that we have two categorical variables in the data. Include them using one-hot encoding.</li>
<li>Fit the model on the training dataset.</li>
<ul>
<li>To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:</li> 
<li>model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)</li>
</ul>     
<li>Calculate the accuracy on the validation dataset and rount it to 2 decimal digits.</li>
</ul> 
</div>

In [10]:
"""Taking Care of Categorical variables"""
y_train = (df_train['price'] >= 152).values.astype(int)
y_val = (df_val['price'] >= 152).values.astype(int)
y_test = (df_test['price'] >= 152).values.astype(int)

del df_train['price']
del df_val['price']
del df_test['price']

numerical.remove('price')

decision = (y_val >= 152).astype(int)


In [11]:
def calculate_accuracy(features):
    # one-hot encoding datasets
    dv = DictVectorizer(sparse=False)

    train_dict = df_train[features].to_dict(orient='records')
    val_dict = df_val[features].to_dict(orient='records')

    X_train = dv.fit_transform(train_dict)
    X_val = dv.transform(val_dict)

    """Fitting the Model on Training Set"""
    model = LogisticRegression(solver='liblinear', C=1.0, random_state=42)
    model.fit(X_train, y_train)

    """Using the model on validation"""
    y_pred = model.predict_proba(X_val)[:,1]

    """Setting up Decision Threshold to 0.5"""
    decision = (y_pred >= 0.5)

    """Calculating accuracy"""
    accuracy = (y_val == decision).mean()
    

    df_pred = pd.DataFrame()
    df_pred['probability'] = y_pred
    df_pred['prediction'] = decision
    df_pred['actual'] = y_val
    df_pred['correct'] = df_pred.prediction == df_pred.actual
    return accuracy, df_pred
acc, df_pred = calculate_accuracy(numerical+categorical)    

print(acc)
df_pred.head()

In [12]:
all_vars_accuracy,_ = calculate_accuracy(numerical+categorical)
all_vars_accuracy.round(2)

<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Question 5:
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>We have 9 features: 7 numerical features and 2 categorical.</li>
<li>Let's find the least useful one using the feature elimination technique.</li>
<li>Train a model with all these features (using the same parameters as in Q4).</li>
<li>Now exclude each feature from this set and train a model without it. Record the accuracy for each model.</li>
<li>For each feature, calculate the difference between the original accuracy and the accuracy without the feature.</li>
<li>Which of following feature has the smallest difference? </li>
<ul>
<li>neighbourhood_group</li> 
<li>room_type</li>
<li>number_of_reviews</li> 
<li>reviews_per_month</li>
</ul> 
</ul>
</div>


In [13]:
useful_features = numerical + categorical
diff = {}
for i in useful_features:
  features = useful_features.copy()
  features.remove(i)
  acc,_ = calculate_accuracy(features)
  diff["Difference in accuracy when removing %s"%i] = all_vars_accuracy - acc

diff

In [14]:
pd.DataFrame(diff.values(), index = diff.keys()).rename({0: 'differences'}, axis = 1).sort_values(by = 'differences', ascending = True).style.background_gradient('Reds')

<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Smallest Difference:
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>number_of_reviews (0.000895)</li> 
</ul></div>

<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Question 6:
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>For this question, we'll see how to use a linear regression model from Scikit-Learn</li>
<li>We'll need to use the original column 'price'. Apply the logarithmic transformation to this column.</li>
<li>Fit the Ridge regression model on the training data.</li>
<li>This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]</li>
<li>Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.</li>
</ul>
</div>


In [15]:
"""Preparing Data for Linear Regression with Price Variable included"""
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)
len(df_train), len(df_val), len(df_test)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

"""Creating List of Numerical and Categorical columns"""
categorical = [col for col in df.columns if df[col].dtype == 'object']
numerical = [col for col in df.columns if col not in categorical]

"""Apply the log transformation to the price variable using the np.log1p() function."""
y_train = np.log1p(df_train['price'].values)
y_val = np.log1p(df_val['price'].values)
y_test = np.log1p(df_test['price'].values)


"""Make sure that the target value ('price') is not in your dataframe."""
del df_train['price']
del df_val['price']
del df_test['price']

"""Taking care of Categorical Variables"""
dv = DictVectorizer(sparse=False)
features = categorical + numerical
features.remove('price')
train_dict = df_train[features].to_dict(orient='records')
val_dict = df_val[features].to_dict(orient='records')

X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)

In [16]:
scores = {}
for alpha in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=alpha, random_state= 42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = np.sqrt(mean_squared_error(y_val, y_pred))
    scores[alpha] = score.round(3)
    print("RMSE with alpha = %s and not rounding to 3 digits: %s"%(alpha, score) )
print(" \nRMSE with rounding off to 3 digits")
print(scores)



In [17]:
print("Table of RMSE rounded to 3 digits")
pd.DataFrame(scores.values(), index = scores.keys()).rename({0: 'RMSE'}, axis = 1).sort_values(by = 'RMSE', ascending = True).style.background_gradient('Reds')

<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:white">$ \large alpha$ that leads to the best RMSE: 0</div>